https://www.thepythoncode.com/article/yolo-object-detection-with-opencv-and-pytorch-in-python

In [1]:
import cv2
import numpy as np

import time
import sys
import os

In [3]:
CONFIDENCE = 0.5
SCORE_THRESHOLD = 0.5
IOU_THRESHOLD = 0.5

config_path = "yolov3-tiny.cfg"        # the neural network configuration
weights_path = "yolov3-tiny.weights"   # the YOLO net weights file

labels = open("coco.names").read().strip().split("\n")        # loading all the class labels (objects)
colors = np.random.randint(0, 255, size=(len(labels), 3), dtype="uint8") # generating colors for each object for later plotting

# Load the Model

In [19]:
# Opencv.dnn: https://learnopencv.com/deep-learning-with-opencvs-dnn-module-a-definitive-guide/
# load the YOLO network
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)

# Image Preparation

In [5]:
path_name = "E:\\Miscelleneous\\street.jpg"
image = cv2.imread(path_name)
file_name = os.path.basename(path_name)
filename, ext = file_name.split(".")

Normalize, scale and reshape this image to be suitable as an input to the neural network.

This will normalize pixel values to range from 0 to 1, resize the image to (416, 416) and reshape it.

In [6]:
h, w = image.shape[:2]
# create 4D blob
blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)

In [7]:
print("image.shape:", image.shape)
print("blob.shape:", blob.shape)

image.shape: (1200, 1800, 3)
blob.shape: (1, 3, 416, 416)


# Making Predictions

In [8]:
net.setInput(blob)        # sets the blob as the input of the network
ln = net.getLayerNames()  # get all the layer names
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Feed forward (inference) and get the network output
# measure how much it took in seconds
start = time.perf_counter()
layer_outputs = net.forward(ln)
time_took = time.perf_counter() - start
print(f"Time took: {time_took:.2f}s")

Time took: 0.57s


In [ ]:
font_scale = 1
thickness = 1
boxes, confidences, class_ids = [], [], []

# loop over each of the layer outputs
for output in layer_outputs:
        
    for detection in output:    # loop over each of the object detections
        
        scores = detection[5:]
        class_id = np.argmax(scores)   # Extract the class id (label) and confidence (as a probability) 
        confidence = scores[class_id]  # of the current object detection
        
        # Discard out weak predictions by ensuring the detected probability is greater than the min probability
        if confidence > CONFIDENCE:
            # scale the bounding box coordinates back relative to the
            # size of the image, keeping in mind that YOLO actually
            # returns the center (x, y)-coordinates of the bounding
            # box followed by the boxes' width and height
            box = detection[:4] * np.array([w, h, w, h])
            (centerX, centerY, width, height) = box.astype("int")
            # use the center (x, y)-coordinates to derive the top and
            # and left corner of the bounding box
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))
            # update our list of bounding box coordinates, confidences,
            # and class IDs
            boxes.append([x, y, int(width), int(height)])
            confidences.append(float(confidence))
            class_ids.append(class_id)

In [10]:
print(detection.shape)

(85,)


# Drawing Detected Objects

In [11]:
# loop over the indexes we are keeping
for i in range(len(boxes)):
    # extract the bounding box coordinates
    x, y = boxes[i][0], boxes[i][1]
    w, h = boxes[i][2], boxes[i][3]
    # draw a bounding box rectangle and label on the image
    color = [int(c) for c in colors[class_ids[i]]]
    cv2.rectangle(image, (x, y), (x + w, y + h), color=color, thickness=thickness)
    text = f"{labels[class_ids[i]]}: {confidences[i]:.2f}"
    # calculate text width & height to draw the transparent boxes as background of the text
    (text_width, text_height) = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, fontScale=font_scale, thickness=thickness)[0]
    text_offset_x = x
    text_offset_y = y - 5
    box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 2, text_offset_y - text_height))
    overlay = image.copy()
    cv2.rectangle(overlay, box_coords[0], box_coords[1], color=color, thickness=cv2.FILLED)
    # add opacity (transparency to the box)
    image = cv2.addWeighted(overlay, 0.6, image, 0.4, 0)
    # now put the text (label: confidence %)
    cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=font_scale, color=(0, 0, 0), thickness=thickness)

In [12]:
cv2.imwrite(filename + "_yolo3." + ext, image)

True

# Non-Maximal Suppression

In [13]:
# perform the non maximum suppression given the scores defined before
idxs = cv2.dnn.NMSBoxes(boxes, confidences, SCORE_THRESHOLD, IOU_THRESHOLD)

In [16]:
# ensure at least one detection exists
if len(idxs) > 0:
    # loop over the indexes we are keeping
    for i in idxs.flatten():
        # extract the bounding box coordinates
        x, y = boxes[i][0], boxes[i][1]
        w, h = boxes[i][2], boxes[i][3]
        # draw a bounding box rectangle and label on the image
        color = [int(c) for c in colors[class_ids[i]]]
        cv2.rectangle(image, (x, y), (x + w, y + h), color=color, thickness=thickness)
        text = f"{labels[class_ids[i]]}: {confidences[i]:.2f}"
        # calculate text width & height to draw the transparent boxes as background of the text
        (text_width, text_height) = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, fontScale=font_scale,thickness=thickness)[0]
        text_offset_x = x
        text_offset_y = y - 5
        box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 2, text_offset_y - text_height))
        overlay = image.copy()
        cv2.rectangle(overlay, box_coords[0], box_coords[1], color=color, thickness=cv2.FILLED)
        # add opacity (transparency to the box)
        image = cv2.addWeighted(overlay, 0.6, image, 0.4, 0)
        # now put the text (label: confidence %)
        cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
            fontScale=font_scale, color=(0, 0, 0), thickness=thickness)

In [17]:
cv2.imwrite(filename + "_yolo3." + ext, image)

True